# Creating a basic chat experience with context variables

In this example, we show how you can build a simple chat bot by sending and updating context with your requests.

We introduce the Context Variables object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The context is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the context on disk so that you can bring it into your applications.  

In this chat scenario, as the user talks back and forth with the bot, the context gets populated with the history of the conversation. During each new run of the kernel, the context can provide the AI with its variables' content.

In [1]:
!python -m pip install semantic-kernel==0.3.15.dev0 numpy==1.24.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.7/200.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 42.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jsonschema-specifications to determine which version is comp

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion

kernel = sk.Kernel()
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("chat_completion", AzureChatCompletion(deployment, endpoint, api_key))
else:
    #api_key, org_id = sk.openai_settings_from_dot_env()
    api_key, org_id = OPENAI_API_KEY, ""
    kernel.add_chat_service("chat-gpt", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))


Let's define a prompt outlining a dialogue chat bot.

In [4]:
sk_prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function

In [5]:
chat_function = kernel.create_semantic_function(sk_prompt, "ChatBot", max_tokens=2000, temperature=0.7, top_p=0.5)

Initialize your context

In [6]:
context = kernel.create_new_context()
context["history"] = ""

Chat with the Bot

In [7]:
context["user_input"] = "Hi, I'm looking for book suggestions"
bot_answer = await chat_function.invoke_async(context=context)
print(bot_answer)

Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?


Update the history with the output

In [8]:
context["history"] += f"\nUser: {context['user_input']}\nChatBot: {bot_answer}\n"
print(context["history"])


User: Hi, I'm looking for book suggestions
ChatBot: Hi! I'd be happy to help you with book suggestions. What genre or type of books are you interested in?



Keep Chatting!

In [9]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    context["user_input"] = input_text

    # Process the user message and get an answer
    answer = await chat_function.invoke_async(context=context)

    # Show the response
    print(f"ChatBot: {answer}")

    # Append the new interaction to the chat history
    context["history"] += f"\nUser: {input_text}\nChatBot: {answer}\n"

In [11]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Error: (<ErrorCodes.ServiceError: 6>, 'OpenAIChatCompletion failed to complete the chat', RateLimitError(message='Your account is not active, please check your billing details on our website.', http_status=429, request_id=None))


In [ ]:
await chat("that sounds interesting, what is it about?")

In [ ]:
await chat("if I read that book, what exactly will I learn about Greek history?")

In [ ]:
await chat("could you list some more books I could read about this topic?")

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!

In [ ]:
print(context["history"])